# **Preparations for Meeting Simulation**

In [ ]:
import numpy as np
import pandas as pd

import json

from multiprocessing.pool import Pool, ThreadPool
from multiprocessing import Lock

from tqdm.notebook import trange

import copy

In [ ]:
SEEDS = [42, 1234, 1867, 613, 1001]
TIME_LIMIT = 300

In [ ]:
RESULTS_ROOT = "../../02_data/01_simulation_results/"
VEH_LIST_PATH = "../../02_data/veh_list.json"
MEETING_PATH = "../../02_data/03_meeting_data/"

In [ ]:
#reading *test* vehicles:

with open(VEH_LIST_PATH) as f:
    veh_list  = json.load(f)

test_vehicles = veh_list["test_vehs"]

In [ ]:
def combine_commuters(veh_id):
    if veh_id.startswith("carIn"):
        return veh_id.split(":")[0]
    return veh_id

## Creating the combination dataset

In [ ]:
#reading parking simulation data and combine commuter vehicle ids:

#data reading:
p_data = pd.DataFrame()

for s in SEEDS:
    filename = f'{RESULTS_ROOT}/poccup_by_vehs_{s}.csv'
    pf = pd.read_csv(filename)
    pf["seed"] = [s]*len(pf)
    p_data = pd.concat([p_data, pf])

#handling the commuters:
p_data["veh_id"] = p_data["veh_id"].apply(combine_commuters)

In [ ]:
#creating key for the measurements:

def calculate_hash(datarow):
    key = r.parking_id + str(r.time) + str(r.occupancy) + str(r.seed)
    return hash(key)


hashes = []
for i,r in p_data.iterrows():
    hashes.append(calculate_hash(r))
p_data["hash"] = hashes

In [ ]:
p_data.to_csv(f"{MEETING_PATH}/combined_dataset.csv", index=False)